### Import thư viện

In [11]:
import os
import re
import json
import random
import shutil

In [ ]:
import pandas as pd

In [2]:
import spacy
# Tải mô hình ngôn ngữ tiếng Anh từ spaCy
nlp = spacy.load("en_core_web_lg")

---

### Đọc và tiền xử lý dữ liệu cho quá trình train mô hình

In [6]:
path_folder = "../data_summarize/"
files = os.listdir(path_folder)

In [8]:
data_list = []
for file in files:
    with open(f"{path_folder}{file}", "r", encoding="utf-8") as f:
        data_list.append(json.load(f))

In [9]:
df = pd.DataFrame(data_list, index=range(len(data_list)))

In [10]:
df

,news,summary
0,By Paul Donnelley Loner: Iain Merrick was jail...,"Iain Merrick, 39, launched a four-year vendett..."
1,Sat nav failed to mention the 15 kilometres of...,Sat nav failed to mention the 15 kilometres of...
2,By Simon Jones for MailOnline Swansea City are...,Modou Barrow will travel to Swansea this week ...
3,A surfer has been airlifted to hospital after ...,Kevin Swanson was on his board near the Montañ...
4,By Victoria Woollaston and Ted Thornhill PUBLI...,The assassination is carried out using a mobil...
...,...,...
149995,Washington (CNN) -- A second grand jury's deci...,NEW Speaker Boehner said American people deser...
149996,"By Emma Glanfield Eric Craggs, 68, of Stockton...","Eric Craggs, 68, accused of asking for 'Lasers..."
149997,Los Angeles (CNN) -- California Gov. Jerry Bro...,Gov. Jerry Brown will seek a stay of what he c...
149998,"Norfolk, Virginia (CNN)The second mate of the ...","Father: ""I know he went through what he went t..."


In [13]:
def preprocessing(sentence,type):
    doc = nlp(sentence)
    # lemmatization và loại bỏ stopword
    lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    # Loại bỏ dấu câu bằng cách thay thế chúng bằng khoảng trắng, viết thường câu văn.
    lemmatized_text = re.sub(r'[^\w\s]', ' ', lemmatized_text).lower().strip()
    lemmatized_text = re.sub(r'\s+', ' ', lemmatized_text)
    if type == "summary":
        return "<sostok> " + lemmatized_text + " <eostok>"
    return lemmatized_text

In [14]:
# Áp dụng hàm tiền xử lý cho cột 'news' và 'summary'
df['news'] = df['news'].apply(preprocessing, type="news")
df['summary'] = df['summary'].apply(preprocessing, type="summary")

KeyboardInterrupt: 

In [ ]:
df

---

Quá trình train mô hình Tóm Tắt được mô tả kỹ ở hai link dưới đây:
- https://www.kaggle.com/code/vannguyenthi/text-summarization-with-seq2seq-model
- 